In [ ]:
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive
# Downloading the rep; this branch already contains the 20 frame of sample kitti data and 
!git clone -b kitti-colab-demo https://github.com/Sim-on-Wheels/Sim-on-wheels-Renderer.git
%cd /content/drive/MyDrive/Sim-on-wheels-Renderer/

In [ ]:
import os
os.listdir()

In [ ]:
!pip install -r requirements.txt

In [1]:
import numpy as np
from omegaconf import OmegaConf, DictConfig
from kitti360 import HIL_rendering, custom_run_window_config, DEFAULT_ARGS
import moderngl_window as mglw
BASE_KITTI_CONFIG_PATH = 'configs/kitti360.yaml'

pygame 2.1.2 (SDL 2.0.18, Python 3.9.12)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
def gen_random_actor_cfg(n_actors: int):
    cfg = {}
    for i in range(n_actors):
        id = 'actor_' + repr(i)
        anim_types = [ 'man_running', 'woman_walking' ] # folder name of the asset from ./assets/human_animations
        animation_type =  str(np.random.choice(anim_types).item())
        x_val = np.random.uniform(-10.0, 5.0) # ranges based on kitti sidewalk limits
        y_val = 0.0 # how high above ground
        z_val = np.random.uniform(4.0, 12.0) # how far away from ego vehicle
        pos = [x_val, y_val, z_val]
        if x_val < -2.5: # actor to the left of the car --> make angle so that actor approximately faces right on average
            rot_angle = np.random.uniform(30, 150)
        else:
            rot_angle = np.random.uniform(-150, -45)
        rot = [0.0, rot_angle, 0.0]
        scale = [2.5, 2.5, 2.5]
        cfg[id] = {
                'animation_type': animation_type,
                'object2scenario': {
                    'pos' : pos,
                    'rot' : rot,
                    'scale' : scale
                }
            }
    return cfg
def auto_gen_kitti_config() -> DictConfig:
    base_cfg = OmegaConf.load(BASE_KITTI_CONFIG_PATH)
    actor_gen = gen_random_actor_cfg(np.random.randint(1, 3))
    base_cfg.scenario_configs.jaywalking.actor_configs = actor_gen
    base_cfg.scenario_configs.jaywalking.trigger_distance = 2.0 # just force immediate animation with low trigger distance
    print(base_cfg.scenario_configs.jaywalking.actor_configs)
    base_cfg.device = 'cpu'
    base_cfg.display = 'headless'
    return base_cfg

In [3]:
custom_config = auto_gen_kitti_config()
config_cls = HIL_rendering
config_cls.argv = DEFAULT_ARGS
values = DEFAULT_ARGS
custom_run_window_config(custom_config, config_cls, values)

{'actor_0': {'animation_type': 'man_running', 'object2scenario': {'pos': [4.848933100407052, 0.0, 10.460488606090006], 'rot': [0.0, -124.06663056109053, 0.0], 'scale': [2.5, 2.5, 2.5]}}}


ImportError: cannot import name 'egl' from 'glcontext' (c:\Users\Bharg\miniconda3\envs\sow\lib\site-packages\glcontext\__init__.py)